
# Testing OpenAI's Whisper STT
By Ben for playground fun

## What To Expect

This notebook allows you to play with Whisper by first creating your own audio recording, processing the audio into a format the model will understand, and feeding it into Whisper for translation or transcription! Let's get started.




## Installs and imports
The commands below will install the Python packages needed to record audio snippets and use Whisper models for speech-to-text transcription.

In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install sounddevice wavio
! pip install ipywebrtc notebook

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-dhoj5r1a
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-dhoj5r1a
  Resolved https://github.com/openai/whisper.git to commit 173ff7dd1d9fb1c4fddea0d41d704cfefeb8908c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.4 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803556 sha256=92673902799f26ad79cdbe3c86f526c6565f57983a477323d95cc6157604b394
  Stored in directory: /tmp/pip-ephem-wheel-cache-pj9odo86/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 8.2 MB/s eta 0:00:0

We also need the following in order to record audio from this notebook and process the resulting files.

In [ ]:
!apt install ffmpeg
!apt-get install libportaudio2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 65.3 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Fetched 65.3 kB in 1s (94.5 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Setting up libportaudio2:amd64 (19.6.0-1.1) ...
Processing triggers for

In [ ]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio, display
import ipywidgets as widgets

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Make your recording

First, we need to enable some Colab widgets so that we can make an audio recording.

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

### Time to record!

Press the circle button and start speaking. It may not look it, but the widget will be capturing sound. Click the circle button again when you are finished. The widget will immediately begin to play back what it captured.

In [ ]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

The audio format captured above is not readable by PyTorch. In this step, we convert our recording into a format that PyTorch can understand.

In [ ]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f wav my_recording.wav -y -hide_banner -loglevel panic

### Alternatively...
If you don't want to make your own recording, you can instead upload an audio file to this notebook.


## Select options

Whisper is capable of performing transcriptions for many languages (though it performs better for some languages and worse for others.)

Whisper is also capable of detecting the input language. However, to be on the safe side, we can explicitly tell Whisper which language to expect.

In [ ]:
language_options = whisper.tokenizer.TO_LANGUAGE_CODE
language_list = list(language_options.keys())

In [ ]:
lang_dropdown = widgets.Dropdown(options=language_list, value='english')
output = widgets.Output()
display(lang_dropdown)

Dropdown(options=('english', 'chinese', 'german', 'spanish', 'russian', 'korean', 'french', 'japanese', 'portu…

In [ ]:
task_dropdown = widgets.Dropdown(options=['transcribe', 'translate'], value='transcribe')
output = widgets.Output()
display(task_dropdown)

Dropdown(options=('transcribe', 'translate'), value='transcribe')

## Load Whisper model

Whisper comes in five model sizes, four of which also have an optimized English-only version. This notebook loads "base"-sized models (bigger than "tiny" but smaller than the others), which require about 1GB of RAM.

If you selected English above, the cell below will load the optimized English-only version. Otherwise, it will load the multilingual model.


In [ ]:
if lang_dropdown.value == "english":
  model = whisper.load_model("base.en")
else:
  model = whisper.load_model("base")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 92.0MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Model is English-only and has 71,825,408 parameters.


Finally, let's set the rest of our task and language options below and see what we've got.

In [ ]:
options = whisper.DecodingOptions(language=lang_dropdown.value, task=task_dropdown.value, without_timestamps=True)
options

DecodingOptions(task='transcribe', language='english', temperature=0.0, sample_len=None, best_of=None, beam_size=None, patience=None, length_penalty=None, prompt=None, prefix=None, suppress_tokens='-1', suppress_blank=True, without_timestamps=True, max_initial_timestamp=1.0, fp16=True)

## Take Whisper for a test drive

All that's left to do now is feed our audio into Whisper.

The cell below performs the last processing steps to make this happen. First, it loads our PyTorch-ready audio file. Then it pads the audio into 30 sec segments. It creates a log-mel spectrogram of the audio and this is fed into Whisper along with the options we set above.



_Note: if you chose to upload your own audio file rather than create one through this notebook, you'll need to update the audio filename below._

In [ ]:
audio = whisper.load_audio("my_recording.wav")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)
result = model.decode(mel, options)

In [ ]:
result.text

'Hi, I need to go to Austin, Texas and then I need to fly to Abu Dhabi and then I need to fly to New York and then I need to fly to London and then I need to fly to Montana and then I need to fly to Indiana.'